In [2]:
import numpy as np
import pandas as pd
import csv
import re
from Bio import SeqIO
import os

# create GO-list tab delimeted  from .obo format

In [75]:
file = open("GOlist.obo", "r")

In [76]:
GO_df = pd.DataFrame()
_id = ""
name = ""
namespace = ""
_def = ""
synonym = []
_term = False

In [77]:
for line in file:
    if line == "\n":
        if _term == False:    
            pass
        elif _term == True:
            # save the previous one
            GO_df = GO_df.append(pd.DataFrame({'ID': _id,'name':name , "namespace":namespace ,"definition" : _def}, index=[0]), ignore_index=True)
            # start the new record
            _id = ""
            name= ""
            namespace = ""
            _def = ""
            _term = False
        continue
        
    line = line.replace('\n', '')
    
    if line.startswith('[Term]'):
        _term = True
              
    if line.startswith('id:'):
        x = line.split()
        _id = x[1] # get id 
        
    if line.startswith('name:'):
        x = line.split(" ", 1)
        name = x[1] # get name
    
    if line.startswith('namespace:'):
        x = line.split(" ", 1)
        namespace = x[1] # get namespace
    
    if line.startswith('def:'):
        x = re.findall(r'"([^"]*)"', line)
        _def = x[0] # get def
    
    #Keep the last one
   # if line.startswith('[Typedef]'):
   #     GO_df = GO_df.append(pd.DataFrame({'ID': _id,'name':name , "name_sapce":namespace ,"definition" : _def}, index=[0]), ignore_index=True)
        

In [78]:
# GO_df.loc[4]["definition"]
GO_df

,ID,name,namespace,definition
0,GO:0000001,mitochondrion inheritance,biological_process,"The distribution of mitochondria, including th..."
1,GO:0000002,mitochondrial genome maintenance,biological_process,The maintenance of the structure and integrity...
2,GO:0000003,reproduction,biological_process,The production of new individuals that contain...
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,OBSOLETE. Assists in the correct assembly of r...
4,GO:0000006,high-affinity zinc transmembrane transporter a...,molecular_function,Enables the transfer of zinc ions (Zn2+) from ...
...,...,...,...,...
47205,GO:2001313,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways involving ...
47206,GO:2001314,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways resulting ...
47207,GO:2001315,UDP-4-deoxy-4-formamido-beta-L-arabinopyranose...,biological_process,The chemical reactions and pathways resulting ...
47208,GO:2001316,kojic acid metabolic process,biological_process,The chemical reactions and pathways involving ...


In [81]:
GO_df.to_csv("GOTerms.csv", columns=GO_df.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## Create AMP GO term

In [138]:
## Find GO Id that contains antibacterial,antimicrobial, antifungal, antiviral, defense, immun,  anticancer
_tmp = GO_df[GO_df['definition'].str.contains("antibacterial|antimicrobial|antifungal|antiviral|defense|defence|immune|immunity|immunological|anticancer")]
_tmp

,ID,name,namespace,definition
1085,GO:0001776,leukocyte homeostasis,biological_process,The process of regulating the proliferation an...
1092,GO:0001783,B cell apoptotic process,biological_process,"Any apoptotic process in a B cell, a lymphocyt..."
1110,GO:0001802,type III hypersensitivity,biological_process,An inflammatory response resulting from recogn...
1128,GO:0001820,serotonin secretion,biological_process,The regulated release of serotonin by a cell. ...
1154,GO:0001846,opsonin binding,molecular_function,Interacting selectively and non-covalently wit...
...,...,...,...,...
46440,GO:2000522,positive regulation of immunological synapse f...,biological_process,Any process that activates or increases the fr...
46444,GO:2000526,positive regulation of glycoprotein biosynthet...,biological_process,Any positive regulation of glycoprotein biosyn...
47086,GO:2001191,regulation of gamma-delta T cell activation in...,biological_process,"Any process that modulates the frequency, rate..."
47087,GO:2001192,negative regulation of gamma-delta T cell acti...,biological_process,"Any process that stops, prevents or reduces th..."


In [89]:
# _tmp = GO_df[GO_df['definition'].str.contains("anti|immun|defense")] <-- incase create  super terms

In [139]:
_tmp.to_csv("amp_tmp.csv", columns=GO_df.columns ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

# Check with groud truth

In [3]:
amp_go_df = pd.read_csv('GoTerm/amp_GO.csv', sep=",", header=0,quoting=csv.QUOTE_ALL)  
amp_go_stringent_df = pd.read_csv('GoTerm/amp_stringent_GO.csv', sep=",", header=0,quoting=csv.QUOTE_ALL)
amp_go_stringent_df

,ID,name,namespace,definition
0,GO:0002775,antimicrobial peptide production,biological_process,The synthesis or release of an antimicrobial p...
1,GO:0002776,antimicrobial peptide secretion,biological_process,The regulated release of an antimicrobial pept...
2,GO:0002777,antimicrobial peptide biosynthetic process,biological_process,The chemical reactions and pathways resulting ...
3,GO:0002778,antibacterial peptide production,biological_process,The synthesis or release of an antibacterial p...
4,GO:0002779,antibacterial peptide secretion,biological_process,The regulated release of an antibacterial pept...
5,GO:0002780,antibacterial peptide biosynthetic process,biological_process,The chemical reactions and pathways resulting ...
6,GO:0002781,antifungal peptide production,biological_process,The synthesis or release of an antifungal pept...
7,GO:0002782,antifungal peptide secretion,biological_process,The regulated release of an antifungal peptide...
8,GO:0002783,antifungal peptide biosynthetic process,biological_process,The chemical reactions and pathways resulting ...
9,GO:0045087,innate immune response,biological_process,Innate immune responses are defense responses ...


# Create Truthset (no_ambiguous)
Filter annotation ID that contain ambigious codiing character  

## DECockroach

In [25]:
anno_DECockroach = pd.read_csv('truth/DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]

_fasta = "pep/DECockroach_SRR1552488.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

de_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_de_amp_df= de_df[de_df['ID'].isin(DECockroach_truth_set.prot_id.to_list())]
_de_amp_df

NON_CODE = "B|Z|J|U|O|X"
_de_amp_df = _de_amp_df[~_de_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]

final_df = pd.DataFrame(_de_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/DECockroach_truth_set.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)


In [6]:
# stringent
anno_DECockroach = pd.read_csv('DECockroach_ann_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_DECockroach['all_GO'] = anno_DECockroach['gene_ontology_BLASTX'] + '`' + anno_DECockroach['gene_ontology_BLASTP'] + "`" +  anno_DECockroach['gene_ontology_Pfam']
DECockroach_truth_set = anno_DECockroach[ anno_DECockroach.all_GO.str.contains ('|'.join(amp_go_stringent_df.ID.to_list()))]
DECockroach_truth_set = DECockroach_truth_set[DECockroach_truth_set["prot_id"] != "."]

_fasta = "fasta_pep/DECockroach_SRR1552488.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

de_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_de_amp_df= de_df[de_df['ID'].isin(DECockroach_truth_set.prot_id.to_list())]
_de_amp_df

NON_CODE = "B|Z|J|U|O|X"
_de_amp_df = _de_amp_df[~_de_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]

final_df = pd.DataFrame(_de_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/DECockroach_truth_set.stringent.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

## BAT

In [29]:
anno_BAT = pd.read_csv('truth/BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains('|'.join(amp_go_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]


_fasta = "pep/BAT_SRR12103592.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

bat_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_bat_amp_df= bat_df[bat_df['ID'].isin(BAT_truth_set.prot_id.to_list())]
NON_CODE = "B|Z|J|U|O|X"
_bat_amp_df = _bat_amp_df[~_bat_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]
final_df = pd.DataFrame(_bat_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/BAT_truth_set.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)

In [7]:
# stringent
anno_BAT = pd.read_csv('BAT_annotation_report.txt', sep="\t", header=0,quoting=csv.QUOTE_ALL)  
anno_BAT['all_GO'] = anno_BAT['gene_ontology_BLASTX'] + '`' + anno_BAT['gene_ontology_BLASTP'] + "`" +  anno_BAT['gene_ontology_Pfam']
BAT_truth_set = anno_BAT[ anno_BAT.all_GO.str.contains('|'.join(amp_go_stringent_df.ID.to_list()))]
BAT_truth_set = BAT_truth_set[BAT_truth_set["prot_id"] != "."]

_fasta = "fasta_pep/BAT_SRR12103592.combined.okay.fa.transdecoder.pep"
with open(_fasta) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        sequence.append(str(seq_record.seq))
        lengths.append(len(seq_record.seq))

bat_df = pd.DataFrame(list(zip(identifiers, sequence, lengths)), columns =['ID', 'Sequence', 'length']) 
_bat_amp_df= bat_df[bat_df['ID'].isin(BAT_truth_set.prot_id.to_list())]
NON_CODE = "B|Z|J|U|O|X"
_bat_amp_df = _bat_amp_df[~_bat_amp_df["Sequence"].str.contains(NON_CODE, regex=True)]
final_df = pd.DataFrame(_bat_amp_df["ID"].values, columns=['prot_id'])
final_df.to_csv("truth_no_ambiguous/BAT_truth_set.stringent.csv", columns=["prot_id"] ,sep=",", quotechar='"',index=False,
            quoting=csv.QUOTE_ALL)